## This script processes documents, segments them into pages, calculates the occurrences of certain user-defined keywords, highlights these words with different colours and annotates the document with the page numbers where the keyphrases occur. <br> <br> Viktoria, June 2021

In [1]:
#This script involves convertion between file types. In order to have permission to do this, give jupyter notebook complete disk access.

In [2]:
import os
import pandas as pd
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
import docx2txt
import re
import pdfplumber
import textract
import ocrmypdf
import pluggy
from tqdm import tqdm
import time
from wordcloud import WordCloud
from collections import Counter
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from statistics import mean
from statistics import stdev
from decimal import *
import requests
from bs4 import BeautifulSoup
import textract
import urllib.request, urllib.error, urllib.parse
import ocrmypdf
import pluggy
import pdfplumber
from tqdm import tqdm
from io import BytesIO
from docx2pdf import convert
import pdfkit
import PyPDF2
from PyPDF2 import PdfFileReader
import fitz
import stamper
from PDFNetPython3 import *
import seaborn as sns
import textwrap
import sys
from datetime import datetime
import codecs
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
import datefinder
import datetime
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from textwrap import wrap
from itertools import chain

#import custom functions
from segment_pdf import segment_pdf
from segment_word import segment_word
from segment_html import segment_html

pd.options.display.max_rows = 1000
pd.options.mode.chained_assignment = None

In [3]:
base = '/Users/Viktoria/Desktop/NLP_AnnotatePdf'
raw = os.path.join(base, 'Raw_data')
preproc = os.path.join(base, 'Preprocessed_data')
results = os.path.join(base, 'Results')

os.chdir(base)

In [4]:
#pip install -r requirements.txt

In [5]:
#pip install git+https://github.com/jbarlow83/OCRmyPDF.git

### Step 1. Access the raw data ('Source_List.csv'). This is a list of titles and URLs to be scraped.

In [6]:
#This is a messy table where text data and URLs are mixed. Access everything and bring them to a clean format.
os.chdir(raw)

df = pd.read_csv('Source_List.csv')

print('N = ', len(df))
df = df.fillna('')
df.head()

N =  485


,Source,Document,URL,Text,Date
0,Australia AUSTRAC,Money laundering/terrorism financing risk asse...,,,30-Mar-21
1,Australia AUSTRAC,Fintel Alliance,,Fintel Alliance is an AUSTRAC initiative estab...,11-May-21
2,Australia AUSTRAC,SMR case study examples,,These case study examples demonstrate the impo...,12-May-21
3,Australia AUSTRAC,Coronavirus (COVID-19) – Working with our repo...,,Coronavirus update The global pandemic...,07-Apr-21
4,Australia AUSTRAC,Junket tour operations in Australia risk asses...,,,10-May-21


In [7]:
#Create a tidy title. Special characters in the filename will break the download later.

def prettify_title(title):
    
    #Create a nice and tidy title. Special characters in the title will throw errors.
    name = re.sub(r'\W+', ' ', title)
    name = re.sub(r'.pdf', '', name)
    name = re.sub('^\s*', '', name)
    name = re.sub('\s*$', '', name)
    name = re.sub('[\[\]:?!-/\+=&)(\"\'\*,]', '', name)
    name = name.strip()
    
    return name

In [8]:
df['Document']=df['Document'].apply(prettify_title)

### Step 2. Visit the URLs and get the text. <br> Everything is converted into pdf and broken down into pages, because the density of keyphrases will be calculated on each page.

In [9]:
#Call the segmenting functions for pdf, word and html. Get the text directly if it is already in the dataframe.
#Change download=0 if you don't want the documents to arrive into the pre-processed folder.

def process_document(title, url, text):
    
    try:

        #If the text column is empty, access the URL
        if not text:
            
            response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'})
            soup = BeautifulSoup(response.text, 'html.parser')

            #word
            if '.docx' in url:
                
                content = segment_word(title, response, 1)

            #pdf 
            elif '.pdf' in url or 'PDF' in soup.text[0:50]:

                content = segment_pdf(title, response, 1)
                

            #probably html
            else:
                content = segment_html(title, url, 1)


        #text is already in the dataframe: just write it out as pdf
        elif text:

            #download as pdf
            document = pdfkit.from_string(text, title + '.pdf')

            #read in the text as pages
            content = {}

            with pdfplumber.open(document) as pdf:

                pages = pdf.pages
                for count,page in enumerate(pages):
                    content[count+1] = page.extract_text()
                    
    except:
        content = 'download error'
        print(title, ' ', url)
    
    
    return content

In [10]:
#Now access the urls and get the text all in one go.
os.chdir(preproc)

start = time.time()

df['TextResult'] = [m for m in map(process_document, df.Document, tqdm(df.URL), df.Text)]

end = time.time()
print(end - start)

  0%|          | 1/485 [00:01<13:08,  1.63s/it]

Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                      
Money laundering terrorism financing risk assessments   


  0%|          | 2/485 [00:02<11:37,  1.44s/it]

Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                      
Fintel Alliance   


  1%|          | 3/485 [00:04<11:07,  1.39s/it]

...

Loading pages (1/6)
QFont::setPixelSize: Pixel size <= 0 (0)
QFont::setPixelSize: Pixel size <= 0 (0)
QFont::setPixelSize: Pixel size <= 0 (0)
Counting pages (2/6)                                               
QFont::setPixelSize: Pixel size <= 0 (0)
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
QFont::setPixelSize: Pixel size <= 0 (0)
Done                                                                      


100%|█████████▉| 484/485 [1:11:00<00:07,  7.51s/it]

Loading pages (1/6)
QFont::setPixelSize: Pixel size <= 0 (0)
QFont::setPixelSize: Pixel size <= 0 (0)
QFont::setPixelSize: Pixel size <= 0 (0)
Counting pages (2/6)                                               
QFont::setPixelSize: Pixel size <= 0 (0)
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
QFont::setPixelSize: Pixel size <= 0 (0)
Done                                                                      


100%|█████████▉| 484/485 [1:11:13<00:08,  8.83s/it]

4273.203236103058


In [11]:
df.head()

,Source,Document,URL,Text,Date,TextResult
0,Australia AUSTRAC,Money laundering terrorism financing risk asse...,,,30-Mar-21,download error
1,Australia AUSTRAC,Fintel Alliance,,Fintel Alliance is an AUSTRAC initiative estab...,11-May-21,download error
2,Australia AUSTRAC,SMR case study examples,,These case study examples demonstrate the impo...,12-May-21,download error
3,Australia AUSTRAC,Coronavirus COVID 19 Working with our reportin...,,Coronavirus update The global pandemic...,07-Apr-21,download error
4,Australia AUSTRAC,Junket tour operations in Australia risk asses...,,,10-May-21,download error
